#   Outdoor Wheather

In [47]:
import matplotlib
%matplotlib inline
import numpy as np
import pandas as pd


In [48]:

!pip install openmeteo-requests
!pip install requests-cache retry-requests numpy pandas


import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry
# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)


In [49]:


def make_dataframe(openmeteo_response):

    hourly = openmeteo_response.Hourly()
    _start = pd.to_datetime(hourly.Time(), unit = "s", utc = True)
    _end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True)
    _delta = pd.Timedelta(seconds = hourly.Interval())

    data = {
        "date": pd.date_range(_start, _end, freq=_delta, inclusive="left"),
        "temperature_2m": hourly.Variables(0).ValuesAsNumpy(),
        "surface_pressure": hourly.Variables(1).ValuesAsNumpy(),
        "wind_speed_10m": hourly.Variables(2).ValuesAsNumpy(),
        "wind_direction_10m": hourly.Variables(3).ValuesAsNumpy(),
        "wind_gusts_10m": hourly.Variables(4).ValuesAsNumpy(),
    }

    return pd.DataFrame(data).set_index("date")

In [50]:
data

date  temperature_2m  surface_pressure  \
Zurich  0      2010-01-01 00:00:00+00:00          0.8940        942.275085   
        1      2010-01-01 01:00:00+00:00          0.4940        941.824402   
        2      2010-01-01 02:00:00+00:00         -0.0060        941.640625   
        3      2010-01-01 03:00:00+00:00         -0.3060        941.112305   
        4      2010-01-01 04:00:00+00:00         -0.5560        940.593262   
...                                  ...             ...               ...   
Bolzano 124507 2024-03-15 19:00:00+00:00         14.6365        982.346619   
        124508 2024-03-15 20:00:00+00:00         13.6365        982.812622   
        124509 2024-03-15 21:00:00+00:00         12.7865        983.198181   
        124510 2024-03-15 22:00:00+00:00         11.8365        983.764832   
        124511 2024-03-15 23:00:00+00:00         10.6365        983.817932   

                wind_speed_10m  wind_direction_10m  wind_gusts_10m  city_code  \
Zurich  0             1.138420          341.564941        9.000000          1   
        1             1.018234           44.999897        9.000000          1   
        2             1.297998          123.690094        9.360000          1   
        3             1.800000          143.130020        9.360000          1   
        4             1.138420           71.564964       10.080000          1   
...                        ...                 ...             ...        ...   
Bolzano 124507        2.902413          240.255203       16.199999          4   
        124508        3.671294          191.309891       15.119999          4   
        124509        2.189795          189.462250       15.119999          4   
        124510        1.018234          135.000107       10.080000          4   
        124511        1.484318           75.963730        7.559999          4   

                    Time        Date  
Zurich  0       00:00:00  2010-01-01  
        1       01:00:00  2010-01-01  
        2       02:00:00  2010-01-01  
        3       03:00:00  2010-01-01  
        4       04:00:00  2010-01-01  
...                  ...         ...  
Bolzano 124507  19:00:00  2024-03-15  
        124508  20:00:00  2024-03-15  
        124509  21:00:00  2024-03-15  
        124510  22:00:00  2024-03-15  
        124511  23:00:00  2024-03-15  

[498048 rows x 9 columns]

In [51]:
# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": [47.36, 46, 47.26, 46.5],
	"longitude": [8.55, 8.95, 11.39, 11.35],
	"start_date": "2010-01-01",
	"end_date": "2024-03-15",
	"hourly": ["temperature_2m", "surface_pressure", "wind_speed_10m", "wind_direction_10m", "wind_gusts_10m"]
}
data = openmeteo.weather_api(url, params=params)
data = dict(zip(("Zurich", "Lugano", "Innsbruck", "Bolzano"), data))
data = pd.concat({key: make_dataframe(val) for key, val in data.items()})

city_codes = {"Zurich": 1, "Lugano": 2, "Innsbruck": 3, "Bolzano": 4}
data['city_code'] = data.index.get_level_values(0).map(city_codes)

In [52]:
data

temperature_2m  surface_pressure  \
        date                                                          
Zurich  2010-01-01 00:00:00+00:00          0.8940        942.275085   
        2010-01-01 01:00:00+00:00          0.4940        941.824402   
        2010-01-01 02:00:00+00:00         -0.0060        941.640625   
        2010-01-01 03:00:00+00:00         -0.3060        941.112305   
        2010-01-01 04:00:00+00:00         -0.5560        940.593262   
...                                           ...               ...   
Bolzano 2024-03-15 19:00:00+00:00         14.6365        982.346619   
        2024-03-15 20:00:00+00:00         13.6365        982.812622   
        2024-03-15 21:00:00+00:00         12.7865        983.198181   
        2024-03-15 22:00:00+00:00         11.8365        983.764832   
        2024-03-15 23:00:00+00:00         10.6365        983.817932   

                                   wind_speed_10m  wind_direction_10m  \
        date                                                            
Zurich  2010-01-01 00:00:00+00:00        1.138420          341.564941   
        2010-01-01 01:00:00+00:00        1.018234           44.999897   
        2010-01-01 02:00:00+00:00        1.297998          123.690094   
        2010-01-01 03:00:00+00:00        1.800000          143.130020   
        2010-01-01 04:00:00+00:00        1.138420           71.564964   
...                                           ...                 ...   
Bolzano 2024-03-15 19:00:00+00:00        2.902413          240.255203   
        2024-03-15 20:00:00+00:00        3.671294          191.309891   
        2024-03-15 21:00:00+00:00        2.189795          189.462250   
        2024-03-15 22:00:00+00:00        1.018234          135.000107   
        2024-03-15 23:00:00+00:00        1.484318           75.963730   

                                   wind_gusts_10m  city_code  
        date                                                  
Zurich  2010-01-01 00:00:00+00:00        9.000000          1  
        2010-01-01 01:00:00+00:00        9.000000          1  
        2010-01-01 02:00:00+00:00        9.360000          1  
        2010-01-01 03:00:00+00:00        9.360000          1  
        2010-01-01 04:00:00+00:00       10.080000          1  
...                                           ...        ...  
Bolzano 2024-03-15 19:00:00+00:00       16.199999          4  
        2024-03-15 20:00:00+00:00       15.119999          4  
        2024-03-15 21:00:00+00:00       15.119999          4  
        2024-03-15 22:00:00+00:00       10.080000          4  
        2024-03-15 23:00:00+00:00        7.559999          4  

[498048 rows x 6 columns]

In [53]:
data.reset_index(inplace=True)
data['Time'] = data['date'].dt.time
data['Date'] = data['date'].dt.date
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 498048 entries, 0 to 498047
Data columns (total 10 columns):
 #   Column              Non-Null Count   Dtype              
---  ------              --------------   -----              
 0   level_0             498048 non-null  object             
 1   date                498048 non-null  datetime64[ns, UTC]
 2   temperature_2m      498048 non-null  float32            
 3   surface_pressure    498048 non-null  float32            
 4   wind_speed_10m      498048 non-null  float32            
 5   wind_direction_10m  498048 non-null  float32            
 6   wind_gusts_10m      498048 non-null  float32            
 7   city_code           498048 non-null  int64              
 8   Time                498048 non-null  object             
 9   Date                498048 non-null  object             
dtypes: datetime64[ns, UTC](1), float32(5), int64(1), object(3)
memory usage: 28.5+ MB


In [60]:
data

,level_0,date,temperature_2m,surface_pressure,wind_speed_10m,wind_direction_10m,wind_gusts_10m,city_code,Time,Date
0,Zurich,2010-01-01 00:00:00+00:00,0.8940,942.275085,1.138420,341.564941,9.000000,1,00:00:00,2010-01-01
1,Zurich,2010-01-01 01:00:00+00:00,0.4940,941.824402,1.018234,44.999897,9.000000,1,01:00:00,2010-01-01
2,Zurich,2010-01-01 02:00:00+00:00,-0.0060,941.640625,1.297998,123.690094,9.360000,1,02:00:00,2010-01-01
3,Zurich,2010-01-01 03:00:00+00:00,-0.3060,941.112305,1.800000,143.130020,9.360000,1,03:00:00,2010-01-01
4,Zurich,2010-01-01 04:00:00+00:00,-0.5560,940.593262,1.138420,71.564964,10.080000,1,04:00:00,2010-01-01
...,...,...,...,...,...,...,...,...,...,...
498043,Bolzano,2024-03-15 19:00:00+00:00,14.6365,982.346619,2.902413,240.255203,16.199999,4,19:00:00,2024-03-15
498044,Bolzano,2024-03-15 20:00:00+00:00,13.6365,982.812622,3.671294,191.309891,15.119999,4,20:00:00,2024-03-15
498045,Bolzano,2024-03-15 21:00:00+00:00,12.7865,983.198181,2.189795,189.462250,15.119999,4,21:00:00,2024-03-15
498046,Bolzano,2024-03-15 22:00:00+00:00,11.8365,983.764832,1.018234,135.000107,10.080000,4,22:00:00,2024-03-15


In [55]:
data.value_counts()

level_0    date                       temperature_2m  surface_pressure  wind_speed_10m  wind_direction_10m  wind_gusts_10m  city_code  Time      Date      
Bolzano    2010-01-01 00:00:00+00:00  4.138000        959.277893        1.080000        90.000000           6.120000        4          00:00:00  2010-01-01    1
Lugano     2019-06-20 20:00:00+00:00  20.983000       982.204773        4.072935        44.999897           11.520000       2          20:00:00  2019-06-20    1
           2019-06-22 00:00:00+00:00  18.132999       985.199768        8.311245        355.030334          15.840000       2          00:00:00  2019-06-22    1
           2019-06-21 23:00:00+00:00  17.282999       984.430542        5.804825        330.255219          16.559999       2          23:00:00  2019-06-21    1
           2019-06-21 22:00:00+00:00  18.083000       984.322205        6.287130        346.759460          18.000000       2          22:00:00  2019-06-21    1
                                       

In [56]:
data.describe()

,temperature_2m,surface_pressure,wind_speed_10m,wind_direction_10m,wind_gusts_10m,city_code
count,498048.000000,498048.000000,498048.000000,498048.000000,498048.000000,498048.000000
mean,11.733518,971.062439,4.911873,175.848221,19.899115,2.500000
std,8.336873,16.163691,3.250610,115.192352,11.020779,1.118035
min,-21.107498,907.231873,0.000000,0.924034,1.440000,1.000000
25%,5.291500,957.620163,2.811690,60.642342,12.240000,1.750000
50%,11.583000,974.013092,4.334974,180.000000,16.919998,2.500000
75%,18.036499,984.229614,6.130579,276.340088,24.840000,3.250000
max,39.786503,1012.088501,40.938820,360.000000,132.839996,4.000000


In [57]:
data.isnull()

,level_0,date,temperature_2m,surface_pressure,wind_speed_10m,wind_direction_10m,wind_gusts_10m,city_code,Time,Date
0,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...
498043,False,False,False,False,False,False,False,False,False,False
498044,False,False,False,False,False,False,False,False,False,False
498045,False,False,False,False,False,False,False,False,False,False
498046,False,False,False,False,False,False,False,False,False,False


In [58]:
data.isna()

,level_0,date,temperature_2m,surface_pressure,wind_speed_10m,wind_direction_10m,wind_gusts_10m,city_code,Time,Date
0,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...
498043,False,False,False,False,False,False,False,False,False,False
498044,False,False,False,False,False,False,False,False,False,False
498045,False,False,False,False,False,False,False,False,False,False
498046,False,False,False,False,False,False,False,False,False,False


In [59]:
data.info

<bound method DataFrame.info of         level_0                      date  temperature_2m  surface_pressure  \
0        Zurich 2010-01-01 00:00:00+00:00          0.8940        942.275085   
1        Zurich 2010-01-01 01:00:00+00:00          0.4940        941.824402   
2        Zurich 2010-01-01 02:00:00+00:00         -0.0060        941.640625   
3        Zurich 2010-01-01 03:00:00+00:00         -0.3060        941.112305   
4        Zurich 2010-01-01 04:00:00+00:00         -0.5560        940.593262   
...         ...                       ...             ...               ...   
498043  Bolzano 2024-03-15 19:00:00+00:00         14.6365        982.346619   
498044  Bolzano 2024-03-15 20:00:00+00:00         13.6365        982.812622   
498045  Bolzano 2024-03-15 21:00:00+00:00         12.7865        983.198181   
498046  Bolzano 2024-03-15 22:00:00+00:00         11.8365        983.764832   
498047  Bolzano 2024-03-15 23:00:00+00:00         10.6365        983.817932   

        wind_speed_